## Monitoring data quality in third-party models from the AWS Marketplace

**Overview:**

This notebook demonstrates how to configure an Amazon SageMaker Data Quality monitoring schedule for a pre-trained third-party model from the AWS Marketplace.

**Contents:**
- Pre-requisites
- Step 1. Initial setup
    - 1.1 [Import packages and modules](#section_1_1)
    - 1.2 [Define global variables](#section_1_2)
    - 1.3 [Uploading sample datasets to your S3 bucket](#section_1_3)
- Step 2. Create and deploy the model endpoint with data capture
    - 2.1 [Create the model](#section_2_1)
    - 2.2 [Create the endpoint configuration with DataCapture](#section_2_2)
    - 2.3 [Create the model endpoint](#section_2_3)
    - 2.4 [Periodically check if the model's endpoint has changed from 'Creating' to 'InService'](#section_2_4)
- Step 3. Create a baselining job to suggest a set of baseline constraints
    - 3.1 [Create baselining job](#section_3_1)
- Step 4. Setup a monitoring schedule to monitor the data captured for the model's endpoint
    - 4.1 [Create a monitoring schedule](#section_4_1)
- Step 5. Invoking the inference endpoint with anomalous data
    - 5.1 [Initialize a Predictor to make prediction requests to the model's endpoint](#section_5_1)
    - 5.2 [Create a data quality constraint violations](#section_5_2)


**Pre-requisites**

This notebook requires a subscription to the [Propensity-Planning to Buy a House](https://aws.amazon.com/marketplace/pp/prodview-vzofptk4lnxii) model, a pre-trained machine learning model package from AWS Marketplace.

<a id=section_1_1></a>
#### 1.1 Import packages and modules

In [445]:
import boto3
import pandas as pd
import requests
import time
import io

from time import gmtime, strftime

from sagemaker import get_execution_role
from sagemaker import session
from sagemaker import ModelPackage

from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

from sagemaker.model_monitor import CronExpressionGenerator
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

<a id=section_1_2></a>
#### 1.2 Globals

In [446]:
# Get the execution role for the notebook instance.
role = get_execution_role()

# Stores configuration state and allows you to create service clients and resources
session = session.Session()

# Create a low-level client representing the Amazon S3 service
s3_client = boto3.client('s3')

# Create a low-level client representing the Amazon SageMaker service
sm_client = boto3.client('sagemaker')

# Create a low-level client representing the Amazon SageMaker Runtime
smr_client = boto3.client('sagemaker-runtime')

In [447]:
# S3
BUCKET = session.default_bucket() # Update as needed
PREFIX = 'third-party-model-seller-name' # Update as needed

DATASETS = ['baseline.csv', 'data_quality_drift.csv']

S3_DATA_CAPTURE_URI = 's3://{}/{}/datacapture'.format(BUCKET, PREFIX)
S3_BASELINE_DATASET_URI = 's3://{}/{}/train/{}'.format(BUCKET, PREFIX, DATASETS[0])
S3_BASELINE_ANALYSIS_RESULTS_URI = 's3://{}/{}/baselining'.format(BUCKET, PREFIX)
S3_DATA_QUALITY_RPT_URI = 's3://{}/{}/reports'.format(BUCKET, PREFIX)

# Model
MODEL_PACKAGE_ARN = 'arn:aws:sagemaker:us-east-1:865070037744:model-package/planning-to-buy-house-basic-28fcb3ca751705854a7171b255d8ef43'  # Update as needed
MODEL_NAME = 'third-party-model'
MODEL_ENDPOINT = 'third-party-model-endpoint'
MODEL_ENDPOINT_CONFIG = 'third-party-model-endpoint-config'
MODEL_BASELINE_JOB = 'third-party-model-baseline-job'
MODEL_MONITOR_SCHEDULE_NAME = 'third-party-model-data-quality-schedule'
MODEL_MONITOR_INSTANCE_TYPE = 'ml.m4.xlarge'
MODEL_INFERENCE_INSTANCE_TYPE = 'ml.m4.xlarge'
MODEL_INSTANCE_COUNT = 1

# Training and testing dataset url path
TRAINING_DATASET_URI = "https://raw.githubusercontent.com/william-screen/model-monitor/main/third-party-models/prosper/data"

<a id=section_1_3></a>
#### 1.3 Uploading sample datasets to your S3 bucket

In [448]:
# Sample training datasets for this demo
for file_name in DATASETS:
   
    dataset = '{}/{}'.format(TRAINING_DATASET_URI, file_name)

    # Sends a GET request to the specified url
    response = requests.get(dataset, stream=True)

    # S3 folder prefix and key
    key = '{0}/train/{1}'.format(PREFIX, file_name)

    # Upload data to this sessions default S3 bucket
    response = s3_client.put_object(Body = response.content,
                         Bucket = BUCKET,
                         Key = key,
                         ContentType = 'text/csv')
    print(response)

{'ResponseMetadata': {'RequestId': '8BBADDF1EC757740', 'HostId': 'R5sN54Ifk8OvvDqbQrISP9kjqJJlbnux275Snid9az0gR+uKJ1erInCmet/80JSUrexfonXDuOg=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'R5sN54Ifk8OvvDqbQrISP9kjqJJlbnux275Snid9az0gR+uKJ1erInCmet/80JSUrexfonXDuOg=', 'x-amz-request-id': '8BBADDF1EC757740', 'date': 'Fri, 08 Jan 2021 14:34:57 GMT', 'etag': '"4ccecc4c596b5986e71639344ccc9da7"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"4ccecc4c596b5986e71639344ccc9da7"'}
{'ResponseMetadata': {'RequestId': 'C07924B58C15FEF9', 'HostId': '3NRxVuyeX6r9LHHfkAmfITScgoIb56WUCvfl+wCi6DfLPLoncv7d9zfpqLlRRXy91G7SkeD15/M=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '3NRxVuyeX6r9LHHfkAmfITScgoIb56WUCvfl+wCi6DfLPLoncv7d9zfpqLlRRXy91G7SkeD15/M=', 'x-amz-request-id': 'C07924B58C15FEF9', 'date': 'Fri, 08 Jan 2021 14:34:57 GMT', 'etag': '"5d7b9ffce4df799c7c09315baca9f7cb"', 'content-length': '0', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'ETag': '"

<a id=section_2_1></a>
#### 2.1 Create the model

Creates a model in Amazon SageMaker from a model package

In [449]:
# PrimaryContainer parameter
model_container_params = { 
    "ModelPackageName": MODEL_PACKAGE_ARN
}

# Creates a model and returns the ModelArn
response = sm_client.create_model(
    ModelName = MODEL_NAME,
    PrimaryContainer = model_container_params,
    ExecutionRoleArn = role,
    EnableNetworkIsolation = True)

In [450]:
print('>> The ARN of the model is: {}'.format(response['ModelArn']))

>> The ARN of the model is: arn:aws:sagemaker:us-east-1:717870172810:model/third-party-model


<a id=section_2_2></a>
#### 2.2 Create the endpoint configuration with DataCapture

Creates an endpoint configuration that Amazon SageMaker hosting services uses to deploy models.

In [451]:
# Describes the resources that you want Amazon SageMaker to provision
product_variant_params = {
    'VariantName': 'AllTraffic',
    'ModelName': MODEL_NAME,
    'InitialInstanceCount': MODEL_INSTANCE_COUNT,
    'InstanceType': MODEL_INFERENCE_INSTANCE_TYPE
}

# Specifies the configuration of your endpoint for model monitor data capture.
data_capture_params = {   
    'EnableCapture': True,
    'InitialSamplingPercentage': 100,
    'DestinationS3Uri': S3_DATA_CAPTURE_URI,
    'CaptureOptions': [
        {
            'CaptureMode': 'Input'
        },
        {
            'CaptureMode': 'Output'
        }        
    ],
    'CaptureContentTypeHeader': {
        'CsvContentTypes': ['text/csv']
    }
}

# Creates an endpoint configuration and returns the EndpointConfigArn    
response = sm_client.create_endpoint_config(
    EndpointConfigName = MODEL_ENDPOINT_CONFIG,
    ProductionVariants=[
        product_variant_params
    ],
    DataCaptureConfig=data_capture_params
)

In [452]:
print('>> The ARN of the endpoint config is: {}'.format(response['EndpointConfigArn']))

>> The ARN of the endpoint config is: arn:aws:sagemaker:us-east-1:717870172810:endpoint-config/third-party-model-endpoint-config


<a id=section_2_3></a>
#### 2.3 Create the model endpoint

Creates an endpoint using the endpoint configuration

In [453]:
# Creates an endpoint and returns the EndpointArn  
response = sm_client.create_endpoint(
    EndpointName = MODEL_ENDPOINT,
    EndpointConfigName = MODEL_ENDPOINT_CONFIG
)

In [454]:
print('>> The ARN of the endpoint is: {}'.format(response['EndpointArn']))

>> The ARN of the endpoint is: arn:aws:sagemaker:us-east-1:717870172810:endpoint/third-party-model-endpoint


<a id=section_2_4></a>
#### 2.4 Periodically check if the model's endpoint has changed from 'Creating' to 'InService'

In [455]:
%%time

# Initialize
model_endpoint_status = None

# Get the model endpoint status descriptors
response = sm_client.describe_endpoint(
    EndpointName=MODEL_ENDPOINT
)

# Set the Endpoint Status value
model_endpoint_status = response['EndpointStatus']

# Check for status updates every 45 seconds
while model_endpoint_status == 'Creating':
    
    # Pause execution for 45 seconds
    time.sleep(45)
    
    # Get the model endpoint status descriptors
    response = sm_client.describe_endpoint(
        EndpointName=MODEL_ENDPOINT
    )

    # Set the Endpoint Status value
    model_endpoint_status = response['EndpointStatus']
    
    # Print the current status of model endpoint
    print('>> The current status of model endpoint "{0}" is {1}'.format(MODEL_ENDPOINT, model_endpoint_status))

>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of model endpoint "third-party-model-endpoint" is Creating
>> The current status of mod

<a id=section_3_1></a>
#### 3.1 Create a baselining job

In [456]:
# Initializes a Monitor instance
default_model_monitor = DefaultModelMonitor(
    role=role,
    instance_count=MODEL_INSTANCE_COUNT,
    instance_type=MODEL_MONITOR_INSTANCE_TYPE
)

# Append timestamp to job name
job_name = '{}-{}'.format(MODEL_BASELINE_JOB, strftime("%Y-%m-%d-%H-%M-%S", gmtime()))

# Suggest baselines for use with Amazon SageMaker Model Monitoring Schedules
job = default_model_monitor.suggest_baseline(
    job_name = job_name,
    baseline_dataset=S3_BASELINE_DATASET_URI,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=S3_BASELINE_ANALYSIS_RESULTS_URI,
    wait=True,
    logs=False
)
# Wait for job to complete processing
job.wait(logs=False)


Job Name:  third-party-model-baseline-job-2021-01-08-14-44-44
Inputs:  [{'InputName': 'baseline_dataset_input', 'S3Input': {'S3Uri': 's3://sagemaker-us-east-1-717870172810/third-party-model-seller-name/train/baseline.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'S3Output': {'S3Uri': 's3://sagemaker-us-east-1-717870172810/third-party-model-seller-name/baselining', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
......................................................................!!

<a id=section_4_1></a>
#### 4.1 Create a monitoring schedule

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule your execution. You might see your execution start anywhere between the first ~20 minutes after the hour boundary (i.e. 00:00 – 00:20). This is expected and done for load balancing on the backend.

In [457]:
default_model_monitor.create_monitoring_schedule(
    monitor_schedule_name = MODEL_MONITOR_SCHEDULE_NAME,
    endpoint_input = MODEL_ENDPOINT,
    output_s3_uri = S3_DATA_QUALITY_RPT_URI,
    statistics = default_model_monitor.baseline_statistics(),
    constraints = default_model_monitor.suggested_constraints(),
    schedule_cron_expression = CronExpressionGenerator.hourly()
)

In [458]:
# Allow time for processing
time.sleep(30)

# Print the current status
monitor_schedule_details = default_model_monitor.describe_schedule()['MonitoringScheduleStatus']
print('>> The current status of monitoring schedule "{0}" is {1}'.format(MODEL_MONITOR_SCHEDULE_NAME, monitor_schedule_details))

>> The current status of monitoring schedule "third-party-model-data-quality-schedule" is Scheduled


<a id=section_5_1></a>
#### 5.1 Initialize a Predictor

Make prediction requests to the model's endpoint.

In [459]:
# Create predictor endpoint
predictor = Predictor(endpoint_name=MODEL_ENDPOINT, 
                      sagemaker_session=None, 
                      serializer=CSVSerializer())   

In [460]:
def predict(sample, delay=0.5):
   
    # Defensive coding
    if(len(sample) > 0):    

        # Invoke the model's inference endpoint
        response = predictor.predict(data=sample)

        # Decode bytes to string
        response = response.decode('utf-8')
        
        # Suspends execution for # milliseconds
        time.sleep(delay)        

        # Return 
        return response

<a id=section_5_2></a>
#### 5.2 Create data quality constraint violations

In [461]:
for file_name in DATASETS[1:]:
    
    # Set datadrift dataset key name
    key = '{0}/train/{1}'.format(PREFIX, file_name)

    # Download from S3
    datadrift_file_obj = s3_client.get_object(Bucket=BUCKET, Key=key)

    # Save to in-memory binary stream since file is relatively small (< 1 Mb)
    datadrift_file_buf = io.BytesIO(datadrift_file_obj['Body'].read())

    # Convert to Dataframe
    df = pd.read_csv(datadrift_file_buf, header=None, na_filter=False)

    # List values by column to maintain the column dtype
    samples = [df[x].values.tolist() for x in df.columns]

    # Use unpacking operator * to unzip the data
    samples = list(list(x) for x in zip(*samples))

    # Invoke real-time inference endpoint 
    for index, sample in enumerate(samples):

        # Get inference response
        response = predict(sample)

        # Display the model's prediction probability
        print('Sample {0} >> Input: {1}: >> Prediction: {2}'.format(index, sample, response))

Sample 0 >> Input: [1, '3', 22, -0.25, -42.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]: >> Prediction: 0.6004737615585327
Sample 1 >> Input: [1, '3', 21, -0.25, -42.0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]: >> Prediction: 0.5937598347663879
Sample 2 >> Input: [1, '4', 24, -0.25, -42.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]: >> Prediction: 0.6058259606361389
Sample 3 >> Input: [1, '4', 24, -0.25, -42.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]: >> Prediction: 0.6058259606361389
Sample 4 >> Input: [1, '3', 21, -0.25, -42.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]: >> Prediction: 0.5937598347663879
Sample 5 >> Input: [1, '3', 20, -0.25, -42.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]: >> Prediction: 0.5937598347663879
Sample 6 >> Input: [1, '3', 20, -0.25, -42.0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 

#### All set

Now that the monitoring schedule has been created and we've generated some sample anamolous data to cause data drift detection, please return to the Amazon SageMaker Studio to checkout the Monitoring Job Details.

#### Cleanup Resources

In [440]:
# Step 1.
print('Stopping monitoring schedule...')
sm_client.stop_monitoring_schedule(MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME)
time.sleep(30) # allow time for processing
print('Listing monitoring schedules')
sm_client.list_monitoring_schedules(EndpointName=MODEL_ENDPOINT) #["MonitoringScheduleSummaries"][0]["MonitoringScheduleStatus"]

Stopping monitoring schedule...
Listing monitoring schedules


{'MonitoringScheduleSummaries': [{'MonitoringScheduleName': 'third-party-model-data-quality-schedule',
   'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-1:717870172810:monitoring-schedule/third-party-model-data-quality-schedule',
   'CreationTime': datetime.datetime(2021, 1, 7, 19, 52, 53, 420000, tzinfo=tzlocal()),
   'LastModifiedTime': datetime.datetime(2021, 1, 8, 14, 25, 28, 840000, tzinfo=tzlocal()),
   'MonitoringScheduleStatus': 'Stopped',
   'EndpointName': 'third-party-model-endpoint',
   'MonitoringJobDefinitionName': 'data-quality-job-definition-2021-01-07-19-52-53-153',
   'MonitoringType': 'DataQuality'}],
 'ResponseMetadata': {'RequestId': '5c0b2292-04e2-46bd-875e-e5d4e77efe24',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5c0b2292-04e2-46bd-875e-e5d4e77efe24',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '491',
   'date': 'Fri, 08 Jan 2021 14:25:54 GMT'},
  'RetryAttempts': 0}}

In [441]:
# Step 2.
print('Deleting monitoring schedule...')
sm_client.delete_monitoring_schedule(MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME)
time.sleep(30) # allow time for processing
print('Listing monitoring schedules')
sm_client.list_monitoring_schedules(EndpointName=MODEL_ENDPOINT) #["MonitoringScheduleSummaries"][0]["MonitoringScheduleStatus"]

Deleting monitoring schedule...
Listing monitoring schedules


{'MonitoringScheduleSummaries': [],
 'ResponseMetadata': {'RequestId': '0ff1fb1d-a97f-4a80-be4a-bc825ad4a2fe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '0ff1fb1d-a97f-4a80-be4a-bc825ad4a2fe',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '34',
   'date': 'Fri, 08 Jan 2021 14:26:23 GMT'},
  'RetryAttempts': 0}}

In [442]:
# Step 3.
print('Deleting model endpoint...')
sm_client.delete_endpoint(EndpointName=MODEL_ENDPOINT)
time.sleep(30) # allow time for processing
print('Listing model endpoints')
sm_client.list_endpoints(NameContains=MODEL_ENDPOINT)

Deleting model endpoint...
Listing model endpoints


{'Endpoints': [],
 'ResponseMetadata': {'RequestId': '9517dc78-6065-402a-a3b6-53ab74fdd6d3',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9517dc78-6065-402a-a3b6-53ab74fdd6d3',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '16',
   'date': 'Fri, 08 Jan 2021 14:26:53 GMT'},
  'RetryAttempts': 0}}

In [443]:
# Step 4.
print('Deleting model endpoint config...')
sm_client.delete_endpoint_config(EndpointConfigName=MODEL_ENDPOINT_CONFIG)
time.sleep(30) # allow time for processing
print('Listing model endpoint configs')
sm_client.list_endpoint_configs(NameContains=MODEL_ENDPOINT_CONFIG)

Deleting model endpoint config...
Listing model endpoint configs


{'EndpointConfigs': [],
 'NextToken': 'cIws2QhTXUIa8bi8Wquyl9JGMObqcQriSo5vzJinvSeckdX/R3gUtA9+ySVHbJyHiEhHqKKtNgqd9ijVHO7F/OXBPv0d0zK9bZaje4SuQ02H/MumH1AtB9bHj2D5pB0Sj/btPLT9jTCXK1g2A2rvW95/D0/QBsvsR7E91w93CanWAgPTawc0zzfCUR6wyrhxa7Xk4DXcMD5YI9qJWwJ8Gu/0KmtpllQss2tFWqldTtVYmZNPFPKqNL1Oyd6c7dhuNRZTNyiOmXEeQSvqsESy21STYnba86JKX+rqOMmVFpFAgZr81c4f5m0Sw2WauaAjrOuLVnQzJbm60UU8g0uSBMzPfSoMbP0Izza8RjzJivoIFbGzz8W8w1Cvoj9BdP6qKW5ZDoxyII/x/kYFvE+kVIM60dz6JyK3FDyBGo6Ew3U+fprIhK+bGn0oD2xuNmBlRrkhYwDRl9PYvKdQqp1MkIydVAtG2K6+w5zFmyGSzvZXfmQF4TdBwGTotKpM8bC8g833saJE2BE+ixfJHvrMFhxsuosb/jNiwc3+MmcFxt0d4oo1m9KsSZqZH+1+WCyZse27r67RpzhLxKnwTAfz8Zbcj1wsvgRrHGYJZqo3mRIh+rGuHN+i1e2AdW0dU6cdRAsITw==',
 'ResponseMetadata': {'RequestId': 'ce7d0dc9-68bd-4c6a-b4f3-f109b5ea8581',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ce7d0dc9-68bd-4c6a-b4f3-f109b5ea8581',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '685',
   'date': 'Fri, 08 Jan 2021 14:27:24 GMT'},
  'RetryAt

In [444]:
# Step 5.
print('Deleting model...')
sm_client.delete_model(ModelName=MODEL_NAME)
time.sleep(30) # allow time for processing
print('Listing models')
sm_client.list_models(NameContains=MODEL_NAME)

Deleting model...
Listing models


{'Models': [],
 'NextToken': 'cIws2QhTXUIa8bi8Wquyl9JGMObqcQriSo5vzJinvSeckdX/R3gUtA9+ySVHbJyHiEhHqKKtNgqd9ijVHO7F/OXBPv0d0zK9bZaje4SuQ02H/MumH1AtB9bHj2D5pB0Sj/btPLT9jTCXK1g2A2rvW95/D0/QBsvsR7E91w93CanWAgPTawc0zzfCUR6wyrhxa7Xk4DXcMD5YI9qJWwJ8Gu/0KmtpllQss2tFWqldTtVYmZNPFPKqNL1Oyd6c7dhuNRZTNyiOmXEeQSvqsETDyO8QnIBelJvAhn/KWMmu8hqLswsra/K8uqkdnR4tPlio3wtbj3rizl+ZoFaz9yocQBce+FZyZl/mQbvmbbWROU1VtOYEqCUa4iyfrl+OtnYUNoPrrQ7yxL+rcg0f4eTH3uR3xboFACQfsfNuZc/V11apcobB/BQb6U0uaYY2FcZDhEAeO158d5kPgoXVcAaxmVVl8xDeKLPuetmrWWokKnZRCfDt1PdRgKRwUTeu6/CjxnYUB51mjH/mAV48RV+qErSivUqJoXvAzKQwvy53i+yKJSSMSTmDgJ2puASqN8s5io4pIv0gSeVWAspH/h9oF3sPQtMqAp4SLb+oUwykI6ZKxvi1',
 'ResponseMetadata': {'RequestId': '828820e7-e161-46e3-9e6f-84f3bae1bb21',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '828820e7-e161-46e3-9e6f-84f3bae1bb21',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '652',
   'date': 'Fri, 08 Jan 2021 14:27:54 GMT'},
  'RetryAttempts': 0}}